In [1]:
#Import the libraries needed to run the code
import pandas as pd
import sqlalchemy as sa
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy.engine import create_engine
import os

In [2]:
import configparser
from sqlalchemy import create_engine

# Read the configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# Get values from the config file
username = config['database']['username']
password = config['database']['password']
host = config['database']['host']
database = config['database']['database']

# Connect using the config values
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}?charset=utf8mb4")
connection = engine.connect()

In [7]:
#Let's take a peak at our data. This code shows the first 5 records
df = pd.read_sql_query("SELECT * FROM V_Orders",
                       connection)
df.head()

,OrderDate,Salesperson,Amount of Orders,Total Due
0,2016-06-30,Jack Powell,1,12.53
1,2015-06-08,Jack Powell,1,62.65
2,2016-01-26,Jack Powell,1,194.36
3,2015-11-30,Jack Powell,1,244.51
4,2015-11-08,Jack Powell,1,35.80


In [8]:
#Now we'll make a simple bar chart of sales by year
fig = px.bar(df, x='OrderDate', y='Total Due')
fig.show()

In [10]:
%pip install dash -q

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Create a Web-based Dashboard using Dash
import dash
from dash import dcc
from dash import html

fig = go.Figure()

for index, row in df.iterrows():
    df_salesperson = df[df['Salesperson'] == row.Salesperson]
    fig.add_trace(
        go.Scatter(x=list(df_salesperson.OrderDate),
               y=list(df_salesperson['Total Due']),
               name=row.Salesperson,
               text=list(df['Total Due'].round(1)),
               textposition="top left"))

salesperson_list = []
visibility = [False for i in range(len(df['Salesperson']))]
for index, row in df.iterrows():
    visibility = [False for i in range(len(df['Salesperson']))]
    visibility[index] = True
    d = dict(label=row.Salesperson,
          method="update",
        args=[{"visible": visibility},
               {"title": "Order Summary",
                "annotations": []}])

    salesperson_list.append(d)
    

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=salesperson_list,
        )
    ])
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)



In the VSCode browser, go the the Main Menu icon on the far left at the top. Go to Terminal and Choose New Terminal. Ports. In the Terminal Window, go to the Ports tab. Find Port 8080, right click, and choose "Open In Browser" to view the dashboard. 

![Ports View](../images/ports.png)

![Open In Browser](../images/open-in-browser.png)

Below is an image of the dashboard.
![Example Dashbaord](../images/dashboard-example.png)

